### Perform Exploratory Data Analysis (EDA) and discuss the data and what you observe prior to beginning modeling and how impact how to proceed ###

In [288]:
# load the data

import pandas as pd
import numpy as np

df = pd.read_csv('8k_diabetes.csv')

In [287]:
del df

Explore the data and explain why I don't need all the columns should be included

I also need charts to show correlations, and to help understanding the data

In [289]:
# Drop weight, payer_code, medical_specialty because they have a large number of missing values

df = df.drop(['weight','payer_code','medical_specialty'], axis = 1)

In [290]:
#acetohexamide,examide,citoglipton,Troglitazone, glimepiride.pioglitazone, metformin.rosiglitazone ,metformin.pioglitazone 

#In the following columns, there are 2 or 3 classes, however one of them has the majority of values

#Tolbutamide, glipizide.metformin 
#- The great majority of the data is NO, 7998. Only 2 are Steady
df = df.drop(['tolbutamide', 'glipizide.metformin'], axis = 1)
#Tolazamide
#- The great majority of the data is NO, 7999. Only 1 is Steady
df = df.drop(['tolazamide'], axis = 1)
#Miglitol
#- The great majority is NO 7997, Steady 2 and Down 1
df = df.drop(['miglitol'], axis = 1)

#Acarbose
#- The majority is NO 7976, Steady 23 and Up 1
df = df.drop(['acarbose'], axis = 1)

#Chlorpropamide
#- No 7990, Steady 9, Up 1
df = df.drop(['chlorpropamide'], axis = 1)

#Nateglinide
#- No 7962, Steady 36, Down 1, Up 1
df = df.drop(['nateglinide'], axis = 1)

#Repaglinide
#- No 7888, Steady 96, Up 11, Down 5
df = df.drop(['repaglinide'], axis = 1)

#acetohexamide has only one value for all rows
df = df.drop(['acetohexamide'], axis =1)

In [291]:
# Drop all rows where discharge_disposition_id = expired, because it means that the patient died.

### Actually, check if this makes sense.

#df = df.drop(df[df.discharge_disposition_id =='Expired'].index)

In [292]:
### Explain why I use Not Mapped instead of ?

df = df.replace('?', 'Not Mapped')

In [293]:
#df['admission_type_id'].isnull().sum()

# I decided to replace the null values in the following columns because the data on these rows could be important

df['admission_type_id'].fillna("Not Mapped", inplace = True)
df['discharge_disposition_id'].fillna("Not Mapped", inplace = True)
df['admission_source_id'].fillna("Not Mapped", inplace = True)

# In the case of the diagnostic description, I prefer to leave it as "" but not null. 
df['diag_2_desc'].fillna("", inplace = True)
df['diag_3_desc'].fillna("", inplace = True)


### Pre-processed categorical data for use in the model and justified pre-processing method. Note this may be different for each algorithm you try ###

In [294]:
import category_encoders as ce

encoder = ce.CatBoostEncoder()

cat_data=cat_data = df.drop(['age','time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses','diag_1_desc','diag_2_desc','diag_3_desc','readmitted'], axis = 1 )

target = df["readmitted"].astype(int)

encoder.fit(cat_data, target)

cat_data = encoder.transform(cat_data)

In [232]:
df.select_dtypes(include=['object', 'bool']).columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'diag_1', 'diag_2',
       'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin', 'glimepiride',
       'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone',
       'troglitazone', 'examide', 'citoglipton', 'insulin',
       'glyburide.metformin', 'glimepiride.pioglitazone',
       'metformin.rosiglitazone', 'metformin.pioglitazone', 'change',
       'diabetesMed', 'readmitted', 'diag_1_desc', 'diag_2_desc',
       'diag_3_desc'],
      dtype='object')

In [295]:
### Use the cat encoder to replace the values from the categories

#encode columns with categorical data

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

cat_data=cat_data = df.drop(['time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses','diag_1_desc','diag_2_desc','diag_3_desc','readmitted'], axis = 1 )

for i in cat_data:
    cat_data[i] = le.fit_transform(cat_data[i])

### Pre-processed numerical data appropriately including handling missing data and justified methods used. Note this may be different for each algorithm you try.

In [187]:
cddat_data

,race,gender,admission_type_id,discharge_disposition_id,admission_source_id,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,...,troglitazone,examide,citoglipton,insulin,glyburide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed
0,0.419789,0.398238,0.374035,0.393524,0.383751,0.044139,0.066208,0.339725,0.399465,0.401848,...,0.39725,0.39725,0.39725,0.384731,0.396933,0.39725,0.39725,0.39725,0.371185,0.345947
1,0.419789,0.398238,0.390943,0.393524,0.383751,0.079450,0.133250,0.445132,0.399465,0.401848,...,0.39725,0.39725,0.39725,0.384731,0.396933,0.39725,0.39725,0.39725,0.371185,0.345947
2,0.419789,0.396094,0.340455,0.504269,0.408601,0.308841,0.462732,0.428881,0.359722,0.401848,...,0.39725,0.39725,0.39725,0.383807,0.396933,0.39725,0.39725,0.39725,0.371185,0.414197
3,0.341096,0.398238,0.398689,0.393524,0.224381,0.425750,0.246575,0.319059,0.399465,0.401848,...,0.39725,0.39725,0.39725,0.383807,0.396933,0.39725,0.39725,0.39725,0.371185,0.414197
4,0.341096,0.398238,0.398689,0.393524,0.433853,0.296168,0.384696,0.079450,0.399465,0.401848,...,0.39725,0.39725,0.39725,0.383807,0.396933,0.39725,0.39725,0.39725,0.432154,0.414197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0.419789,0.396094,0.374035,0.439829,0.383751,0.461694,0.336239,0.481570,0.399465,0.401848,...,0.39725,0.39725,0.39725,0.383807,0.396933,0.39725,0.39725,0.39725,0.371185,0.414197
7996,0.419789,0.398238,0.390943,0.393524,0.433853,0.399542,0.389450,0.399750,0.399465,0.401848,...,0.39725,0.39725,0.39725,0.383807,0.396933,0.39725,0.39725,0.39725,0.432154,0.414197
7997,0.419789,0.396094,0.398689,0.378326,0.408601,0.368818,0.374107,0.428881,0.366780,0.401848,...,0.39725,0.39725,0.39725,0.383807,0.396933,0.39725,0.39725,0.39725,0.432154,0.414197
7998,0.419789,0.396094,0.390943,0.439829,0.433853,0.525743,0.399522,0.492272,0.399465,0.401848,...,0.39725,0.39725,0.39725,0.442241,0.396933,0.39725,0.39725,0.39725,0.432154,0.414197


In [296]:
df['age'] = df['age'].replace({"[70-80)":75,
                         "[60-70)":65,
                         "[50-60)":55,
                         "[80-90)":85,
                         "[40-50)":45,
                         "[30-40)":35,
                         "[90-100)":95,
                         "[20-30)":25,
                         "[10-20)":15,
                         "[0-10)":5})

In [297]:
# This code will return all the numeric columns

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

num_data = df.select_dtypes(include=numerics)

In [298]:
target_data = df["readmitted"].astype(int)

In [299]:
import nltk
from time import time
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn import metrics

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support

from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import GradientBoostingClassifier



In [163]:
df['age']

0       55
1       25
2       85
3       55
4       55
        ..
7995    75
7996    55
7997    55
7998    85
7999    75
Name: age, Length: 8000, dtype: int64

### Lemmatize the text fields

In [300]:
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

# clean_text is a function to remove tokens like white spaces in the text

def clean_text(text):
    text = re.sub(r'[^A-Za-z0-9]+',' ',text)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text


In [312]:
## Clean the stop words from the text

df['Processed_diag_1_desc'] = df.diag_1_desc.apply(lambda x: clean_text(x))

df['Processed_diag_2_desc'] = df.diag_2_desc.apply(lambda x: clean_text(x))

df['Processed_diag_3_desc'] = df.diag_3_desc.apply(lambda x: clean_text(x))

text_data = pd.DataFrame(text_data)


In [313]:
# join the processed dataframes
X = pd.concat([cat_data,num_data,text_data],axis=1)
y = target_data

Split Data

#### Apply TfidfVectorizer and Make Predictions


In [314]:
X_train, X_test, y_train, y_test = train_test_split(X[['Processed_diag_1_desc','Processed_diag_2_desc','Processed_diag_3_desc']], y, random_state = 0)

In [304]:
from sklearn.compose import ColumnTransformer

text_columns = ['Processed_diag_1_desc','Processed_diag_2_desc','Processed_diag_3_desc']

tfidf_params = dict(sublinear_tf= True, 
                       #min_df = 5, 
                       norm= 'l2', 
                       #ngram_range= (1,2), 
                       stop_words ='english')


In [315]:
# get the params
tfidf_params = dict(sublinear_tf= True, 
                       #min_df = 5, 
                       norm= 'l2', 
                       #ngram_range= (1,2), 
                       stop_words ='english')

# create a Pipeline that will do features transformation then pass to the model

cls = Pipeline(steps=[
#('features', TfidfVectorizer(**tfidf_params)),
#('elasticnet', ElasticNet(random_state=0))
('clf', LogisticRegression(solver="saga", penalty="elasticnet", l1_ratio=0.5))
])



In [244]:
X_train

,Processed_diag_1_desc,Processed_diag_2_desc,Processed_diag_3_desc
286,cellulitis abscess face,diabetes mellitus without mention complication...,hyperosmolality hypernatremia
6696,postoperative shock unspecified,hyperosmolality hypernatremia,sideroblastic anemia
6486,paroxysmal supraventricular tachycardia,congestive heart failure unspecified,coronary atherosclerosis unspecified type vess...
1145,cellulitis abscess face,phlebitis thrombophlebitis superficial vessel ...,diabetes renal manifestation type juvenile typ...
6346,abnormal glandular papanicolaou smear cervix,chronic kidney disease ckd,congestive heart failure unspecified
...,...,...,...
4931,urinary tract infection site specify,pressure ulcer unspecified site,pressure ulcer unspecified site
3264,streptococcal septicemia,pneumonitis due inhalation food vomitus,acute kidney failure
1653,diabetes renal manifestation type ii unspecifi...,hyperosmolality hypernatremia,nephrotic syndrome lesion proliferative glomer...
2607,acute cor pulmonale,malignant essential hypertension,pure hypercholesterolemia


In [306]:
X_test['Processed_diag_1_desc'].isna().sum()a

45

In [309]:

# Use clf as a model, fit X_train and y_train
cls.fit(X_train['Processed_diag_1_desc'], y_train)
# predicted 
y_pred_model1 = cls.predict(X_test['Processed_diag_1_desc'])

ValueError: could not convert string to float: 'cellulitis abscess face'

In [310]:
# Use clf as a model, fit X_train and y_train
cls.fit(X_train['Processed_diag_2_desc'], y_train)

# predicted 
y_pred_model2 = cls.predict_proba(X_test['Processed_diag_2_desc'])

ValueError: could not convert string to float: 'diabetes mellitus without mention complication type ii unspecified type state uncontrolled'

In [217]:
X_train['Processed_diag_1_desc']

286                               cellulitis abscess face
6696                      postoperative shock unspecified
6486              paroxysmal supraventricular tachycardia
1145                              cellulitis abscess face
6346         abnormal glandular papanicolaou smear cervix
                              ...                        
4931                 urinary tract infection site specify
3264                             streptococcal septicemia
1653    diabetes renal manifestation type ii unspecifi...
2607                                  acute cor pulmonale
2732                               ni disease unspecified
Name: Processed_diag_1_desc, Length: 6000, dtype: object

In [311]:
# Use clf as a model, fit X_train and y_train
cls.fit(X_train['Processed_diag_3_desc'], y_train)

# predicted 
y_pred_model3 = cls.predict_proba(X_test['Processed_diag_3_desc'])

ValueError: could not convert string to float: 'hyperosmolality hypernatremia'

In [246]:
y_train

286     0
6696    0
6486    0
1145    1
6346    1
       ..
4931    1
3264    0
1653    1
2607    0
2732    0
Name: readmitted, Length: 6000, dtype: int32

### Make new predictions with GBT using the text predictions, categorical and numerical data

In [ ]:

# example of using the ColumnTransformer for the Abalone dataset
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
 
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/abalone.csv'
dataframe = read_csv(url, header=None)
# split into inputs and outputs
last_ix = len(dataframe.columns) - 1
X, y = dataframe.drop(last_ix, axis=1), dataframe[last_ix]
print(X.shape, y.shape)
# determine categorical and numerical features
numerical_ix = X.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X.select_dtypes(include=['object', 'bool']).columns
# define the data preparation for the columns
t = [('cat', OneHotEncoder(), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)
# define the model
model = SVR(kernel='rbf',gamma='scale',C=100)
# define the data preparation and modeling pipeline
pipeline = Pipeline(steps=[('prep',col_transform), ('m', model)])
# define the model cross-validation configuration
cv = KFold(n_splits=10, shuffle=True, random_state=1)
# evaluate the pipeline using cross validation and calculate MAE
scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# convert MAE scores to positive values
scores = absolute(scores)
# summarize the model performance
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

clf.fit(X_train, y_train)

#clf.score(X_test, y_test)

In [177]:
X_train

,text_model1,text_model2,text_model3
0,0.638575,0.559031,0.567436
1,0.361425,0.440969,0.432564


In [ ]:
X_train

In [ ]:
text_model1 = pd.DataFrame(y_pred_model1[:],columns = ["text_model1"])
text_model2 = pd.DataFrame(y_pred_model2[:], columns = ["text_model2"])
text_model3 = pd.DataFrame(y_pred_model3[:], columns = ["text_model3"])

num_data_train = pd.DataFrame(num_data.loc[y_train.index])

cat_data_train = pd.DataFrame(cat_data.loc[y_train.index])

# join the new inputs
#X_train = pd.concat([cat_data_train,num_data_train,text_model1,text_model2,text_model3],axis=1)
X_train = pd.concat([text_model1,text_model2,text_model3],axis=1)



In [84]:
fpr, tpr, thresholds = precision_recall_curve(y_test, y_pred_model3[:,1])

NameError: name 'precision_recall_curve' is not defined

In [78]:
y_pred_model2 

array([False, False, False, ..., False, False, False])

In [34]:
X_train

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,diag_1,diag_2,diag_3,max_glu_serum,...,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,text_model1,text_model2,text_model3
0,2.0,0.0,5.0,0.0,1.0,6.0,325.0,291.0,296.0,2.0,...,35.0,4.0,21.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
2,2.0,1.0,8.0,3.0,11.0,5.0,202.0,162.0,77.0,0.0,...,12.0,0.0,21.0,0.0,0.0,1.0,9.0,0.0,0.0,0.0
3,0.0,0.0,5.0,1.0,1.0,9.0,306.0,146.0,39.0,2.0,...,33.0,1.0,5.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0
4,0.0,0.0,5.0,1.0,1.0,2.0,104.0,40.0,97.0,2.0,...,31.0,0.0,13.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7991,2.0,1.0,7.0,3.0,1.0,5.0,171.0,142.0,39.0,3.0,...,10.0,0.0,7.0,0.0,0.0,0.0,5.0,NaN,NaN,NaN
7992,2.0,1.0,5.0,3.0,1.0,6.0,59.0,73.0,404.0,2.0,...,10.0,0.0,4.0,0.0,0.0,0.0,2.0,NaN,NaN,NaN
7993,2.0,1.0,5.0,4.0,18.0,2.0,349.0,142.0,39.0,3.0,...,48.0,0.0,4.0,0.0,0.0,1.0,3.0,NaN,NaN,NaN
7994,2.0,0.0,4.0,4.0,1.0,5.0,58.0,252.0,252.0,2.0,...,88.0,1.0,16.0,0.0,2.0,2.0,8.0,NaN,NaN,NaN
